In [6]:
!pip install moviepy
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-lse3urwn
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-lse3urwn
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803669 sha256=282428b52c74076ac2b7ff2f5b39f75adf82c22074ee320df11ff895c33429e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-u3cupzic/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [7]:
from moviepy.editor import VideoFileClip
import whisper
import cv2
import os

  if event.key is 'enter':



In [8]:
# Function to transcribe audio and generate captions with timestamps
def generate_captions(audio_path, model_name="base"):
    model = whisper.load_model(model_name)
    result = model.transcribe(audio_path, word_timestamps=True, )  # Use word timestamps for better precision
    return result["segments"]

def overlay_captions_on_video(video_path, segments, output_video_path="output_video_with_captions.mp4"):
    # Open video capture
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 file
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    current_caption_index = 0
    segments_len = len(segments)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Get the current time of the video
        current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Convert to seconds

        # Check if the current time matches the segment time range
        while current_caption_index < segments_len and segments[current_caption_index]["end"] < current_time:
            current_caption_index += 1

        if current_caption_index < segments_len:
            segment = segments[current_caption_index]
            # If the current time falls within the caption's time range, overlay the text
            if segment["start"] <= current_time <= segment["end"]:
                # Text and font parameters
                text = segment["text"]
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.3
                color = (0, 0, 0)  # White
                thickness = 1

                # Get the size of the text to adjust the position
                text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
                text_width, text_height = text_size

                # Ensure the text fits within the frame (adjust horizontally and vertically)
                x = 50  # Start at some horizontal offset
                y = height - 50  # Vertical position near the bottom of the frame

                # Check if the text goes out of bounds horizontally and adjust
                if x + text_width > width - 50:
                    x = width - text_width - 50

                # Check if the text goes out of bounds vertically and adjust
                if y - text_height < 50:
                    y = 50 + text_height

                # Overlay the text on the frame
                cv2.putText(frame, text, (x, y), font, font_scale, color, thickness, cv2.LINE_AA)

        # Write the frame with the overlayed caption
        out.write(frame)

    cap.release()
    out.release()
    print(f"Output video saved to {output_video_path}")

In [9]:
def main(video_path, audio_path, model_name="medium", output_video_path="output_video_with_captions.mp4"):
    try:
        # Step 2: Generate captions
        print(f"Generating captions using Whisper ({model_name} model)...")
        segments = generate_captions(audio_path, model_name)

        # Print the captions with timestamps
        print("\nCaptions with Timestamps:")
        captions = ""
        for segment in segments:
            start_time = segment["start"]
            end_time = segment["end"]
            text = segment["text"]
            print(f"[{start_time:.2f}s - {end_time:.2f}s] {text}")
            captions = captions + f"[{start_time:.2f}s - {end_time:.2f}s] {text} "

        # Step 3: Overlay captions on video
        print(f"Overlaying captions on video...")
        overlay_captions_on_video(video_path, segments, output_video_path)

        # Cleanup
        os.remove(audio_path)
        print("Audio file cleaned up.")
        return captions
    except Exception as e:
        print(f"Error: {e}")


In [10]:
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.2 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-f2e48sqo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-f2e48sqo
  Resolved https://github.com/huggingface/transformers.git to commit 15bd3e61f8d3680ca472c9314ad07584d20f7b81
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10450848 sha256=e1e65202a5689afd4423d8e0e7e95b9c1d2e3ee8f95623301c85ed0bec2d6011
  Stored in directory: /tmp/pip-ephem-wheel-cache-w14v4pa8/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:

In [11]:
import torch
from transformers import VideoLlavaProcessor, VideoLlavaForConditionalGeneration, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

model_id = "LanguageBind/Video-LLaVA-7B-hf"

processor = VideoLlavaProcessor.from_pretrained(model_id)
model = VideoLlavaForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



preprocessor_config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/148 [00:00<?, ?B/s]

In [52]:

import av
import numpy as np
from huggingface_hub import hf_hub_download

def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [47]:
def audio_sep(video_file):

  # Load the video file
  video = VideoFileClip(video_file)

  # Extract the audio
  audio = video.audio

  # Save the audio as a .wav file
  audio.write_audiofile("output_audio.wav", codec='pcm_s16le')

  # Close the video and audio objects to free up resources
  audio.close()
  video.close()


In [41]:
# video_path = '/content/sample_data/demo.mp4'

# # Open the video using PyAV
# container = av.open(video_path)

# # sample uniformly 8 frames from the video
# total_frames = container.streams.video[0].frames
# indices = np.arange(0, total_frames, total_frames / 8).astype(int)
# clip = read_video_pyav(container, indices)

# audio_sep(video_path)

# # Call the main function with local file paths
# captions = main(video_file, audio_file, model_name="base", output_video_path="captioned_video.mp4")

MoviePy - Writing audio in output_audio.wav


MoviePy - Done.
Generating captions using Whisper (base model)...


  checkpoint = torch.load(fp, map_location=device)




Captions with Timestamps:
[2.00s - 5.00s]  samawud wab dijghtatw tte kam собствен pen
[9.02s - 14.32s]  oom om'om jouou been nan
Overlaying captions on video...
Output video saved to captioned_video.mp4
Audio file cleaned up.


In [82]:
# Store the previous conversation history as a string (not a list)
conversation_history = "USER: <video>\n"


def get_last_assistant_response(conversation_history):
    # Split the conversation at each "ASSISTANT:"
    parts = conversation_history.split("ASSISTANT:")

    # If there are no "ASSISTANT:" parts, return empty
    if len(parts) < 2:
        return ""

    # Return the last response after the last "ASSISTANT:"
    last_response = parts[-1].strip()
    return last_response


# Define the function to handle context and generate the response
def generate_response_with_context(prompt, video_path, max_length=200):

    global conversation_history

    # Provide a default prompt if the user input is empty or whitespace
    if not prompt.strip():
        return "Empty Prompt"


    # Add the new prompt to the conversation history
    conversation_history += f"{prompt}\nASSISTANT:"

    # Tokenize the prompt
    inputs = processor(conversation_history, videos=clip, return_tensors="pt").to(model.device)

    # Generation configuration
    generate_kwargs = {
        "max_new_tokens": max_length,
        "do_sample": True,
        "top_p": 0.9,
        "top_k": 2
    }

    # Generate the response
    output = model.generate(**inputs, **generate_kwargs)
    generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]

    #get last response ie the text after last 'ASSISTENT'
    text = get_last_assistant_response(generated_text)
    # Add the assistant's response to the conversation history
    conversation_history += f" {text}\nUSER: "

    # Print the generated response
    return f"ASSISTANT:\n {text}"


# Define the function to handle context and generate the response
def process_video(video_path):

    global conversation_history
    global clip

    # Add the new prompt to the conversation history
    conversation_history = f"USER: <video>\n"

    import av
    # Open the video using PyAV
    container = av.open(video_path)

    # sample uniformly 8 frames from the video
    total_frames = container.streams.video[0].frames
    indices = np.arange(0, total_frames, total_frames / 8).astype(int)
    clip = read_video_pyav(container, indices)

    audio_sep(video_path)

    audio_file="/content/output_audio.wav"

    # Call the main function with local file paths
    captions = main(video_path, audio_file, model_name="base", output_video_path="captioned_video.mp4")

    # Print the generated response
    return f"Processing Complete"



In [50]:
# while True:
#     prompt = input("You:\n")
#     video_path = '/content/sample_data/demo.mp4'
#     response = generate_response_with_context(prompt, video_path)
#     print(response)

You:
explain
MoviePy - Writing audio in output_audio.wav


MoviePy - Done.
Generating captions using Whisper (base model)...


  checkpoint = torch.load(fp, map_location=device)




Captions with Timestamps:
[1.68s - 3.24s]   birlikte fishes with a moon
[3.24s - 3.44s]  Snapchat
[3.44s - 3.76s]  ,
[3.76s - 6.62s]  ch Feeling
[6.62s - 6.62s] 
[6.62s - 6.62s] 
[6.62s - 6.66s]  ,
[6.66s - 7.46s]  ,
[7.46s - 7.92s]  ,
[7.92s - 7.92s] 
[7.92s - 7.92s] 
[7.92s - 7.92s] 
[7.92s - 7.92s] 
[7.92s - 8.00s]  ,
[8.00s - 8.18s]  ,
[8.18s - 8.18s] 
[8.18s - 8.18s] 
[8.18s - 8.18s] 
[8.18s - 8.32s]  ,
[8.32s - 8.32s] 
[8.32s - 8.64s]  ,
[8.64s - 8.64s] 
[8.64s - 8.64s] 
[8.64s - 8.88s]  ,
[8.88s - 8.88s] 
[8.88s - 9.60s]  .
[9.60s - 10.90s]  ,
[10.90s - 14.92s]  वूशके तूवववववववववववववववव
Overlaying captions on video...
Output video saved to captioned_video.mp4
Audio file cleaned up.
ASSISTANT:
The scene shows a man and a woman sitting on a couch with a laptop on their laps. They are working on their respective laptops, which are open and placed on their laps. The man is wearing a black shirt and has a black backpack placed next to him. The woman is wearing a blue shirt and has a

KeyboardInterrupt: Interrupted by user

In [ ]:
import gradio as gr
import os

title_markdown = ("""
<div style="display: flex; justify-content: center; align-items: center; text-align: center;">
  <a href="https://github.com/DAMO-NLP-SG/VideoLLaMA2" style="margin-right: 20px; text-decoration: none; display: flex; align-items: center;">
    <img src="https://i.ibb.co/FBsSrLV/img.png" style="max-width: 120px; height: auto;">
  </a>
  <div>
    <h1>Video Query</h1>
  </div>
</div>
""")

block_css = """
#buttons button {
    min-width: min(120px,100%);
    color: #9C276A
}
"""


plum_color = gr.themes.colors.Color(
    name='plum',
    c50='#F8E4EF',
    c100='#E9D0DE',
    c200='#DABCCD',
    c300='#CBA8BC',
    c400='#BC94AB',
    c500='#AD809A',
    c600='#9E6C89',
    c700='#8F5878',
    c800='#804467',
    c900='#713056',
    c950='#662647',
)

def generate_mock_response(audio_video, message, chatbot, textbox_in):

    response = generate_response_with_context(textbox_in, audio_video)
    chatbot.append([textbox_in, response])
    message.append(textbox_in)
    message.append(response)
    return audio_video, message, chatbot

def regenerate(message, chatbot):
    message.pop(-1)
    message.pop(-1)
    chatbot.pop(-1)
    return message, chatbot

def clear_history(message, chatbot):
    message.clear()
    chatbot.clear()
    return (
        gr.update(value=None, interactive=True),
        message, chatbot,
        gr.update(value=None, interactive=True)
    )
def process_video_only(audio_video, message, chatbot):
    video_path = audio_video
    response = process_video(video_path)
    chatbot.append(["[Video Processed]", response])
    message.append(f"[Video Processed]")
    message.append(response)
    return audio_video, message, chatbot

# Theme setup
theme = gr.themes.Default(primary_hue=plum_color)
theme.set(slider_color="#9C276A")
theme.set(block_title_text_color="#9C276A")
theme.set(block_label_text_color="#9C276A")
theme.set(button_primary_text_color="#9C276A")

with gr.Blocks(title='VideoLLaMA 2 🔥🚀🔥', theme=theme, css=block_css) as demo:
    gr.Markdown(title_markdown)
    message = gr.State([])

    with gr.Row():
        with gr.Column(scale=3):
            audio_video = gr.Video(label="Input Video")
            video_submit_btn = gr.Button(value="Process Video", variant="primary", interactive=True)

        with gr.Column(scale=7):
            chatbot = gr.Chatbot(label="VideoLLaMA 2", bubble_full_width=True, height=750)
            with gr.Row():
                with gr.Column(scale=8):
                    textbox = gr.Textbox(show_label=False, placeholder="Enter text and press ENTER", container=False)
                with gr.Column(scale=1, min_width=50):
                    submit_btn = gr.Button(value="Send", variant="primary", interactive=True)
            with gr.Row(elem_id="buttons") as button_row:
                regenerate_btn = gr.Button(value="🔄  Regenerate", interactive=True)
                clear_btn = gr.Button(value="🗑️  Clear history", interactive=True)

    video_submit_btn.click(
        process_video_only,
        inputs=[audio_video, message, chatbot],
        outputs=[audio_video, message, chatbot]
    )


    submit_btn.click(
        generate_mock_response,
        [audio_video, message, chatbot, textbox],
        [audio_video, message, chatbot]
    )

    regenerate_btn.click(
        regenerate,
        [message, chatbot],
        [message, chatbot]
    )

    clear_btn.click(
        clear_history,
        [message, chatbot],
        [audio_video, message, chatbot, textbox]
    )

demo.launch(share=True,debug=True)

  warnings.warn(



Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b2eee00d2ae1837c25.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


MoviePy - Writing audio in output_audio.wav


MoviePy - Done.
Generating captions using Whisper (base model)...


  checkpoint = torch.load(fp, map_location=device)




Captions with Timestamps:
[6.86s - 8.26s] vari
[14.18s - 15.58s]  ,
Overlaying captions on video...
Output video saved to captioned_video.mp4
Audio file cleaned up.
